### Градиентный бустинг (Gradient Boosting)
На прошлой лекции мы говорили про соединение базовых алгоритмов в ансамбль с помощью бэггинга (и, в частности, строить из решающих деревьев случайные леса). Теперь мы рассмотрим другой способ объединять базовые алгоритмы в композицию – градиентный бустинг.

В ходе обучения случайного леса каждый базовый алгоритм строится независимо от остальных. Бустинг, в свою очередь, воплощает идею последовательного построения линейной комбинации алгоритмов. Каждый следующий алгоритм старается уменьшить ошибку текущего ансамбля. Бустинг, использующий деревья решений в качестве базовых алгоритмов, называется **градиентным бустингом над решающими деревьями**, **Gradient Boosting on Decision Trees**, **GBDT**. Он отлично работает на выборках с "табличными", неоднородными данными. Такой бустинг способен эффективно находить нелинейные зависимости в данных различной природы. Этим свойством обладают все алгоритмы, использующие деревья решений, однако именно GBDT обычно выигрывает в подавляющем большинстве задач. Не так хорошо бустинг проявляет себя на однородных данных: текстах, изображениях, звуке, видео. В таких задачах нейросетевые подходы почти всегда демонстрируют лучшее качество.


Бустинг можно представить как гольфиста, цель которого – загнать мяч в лунку с координатой $y_{ball}$. Представим, что для решения мы строим композицию из $K$ базовых алгоритмов $a(x)=a_K(x)=b_1(x)+b_2(x)+\cdots+b_K(x)$. Положение мяча здесь – ответ нашей композиции $a(x_{ball})$. После первого удара гольфист (скорее всего) не попадет в лунку, но зато приблизится к ней. Следующий удар гольфиста переводит мяч из текущего положения $a_k(x_{ball})$  в положение $a_{k+1}(x_{ball})$. Каждый следующий удар – это та поправка, которую вносит очередной базовый алгоритм в композицию. Если гольфист все делает правильно, то функция потерь будет уменьшаться (мяч приближается к лунке):
$$
L\left(y, a_{k+1}(x) \right) < L\left(y, a_{k}(x) \right)
$$

<center><img src="data//boosting_golf.png" alt="drawing" width="800"/></center>

Удары при этом делаются не хаотично. Гольфист оценивает текущее положение мяча относительно лунки и следующим ударом старается нивелировать те проблемы, которые он создал себе всеми предыдущими. Подобно тому, как гольфист постепенно подводит мяч к цели, **бустинг** с каждым новым базовым алгоритмом всё больше приближает предсказание к истинному значению метки объекта.

#### Пример с задачей регрессии
Пусть имеем квадратичную функцию потерь:
$$
L(y, x)=\frac{1}{2}\sum\limits_{i=1}^{N}\left(y_i-a(x_i) \right)^2 \rightarrow min
$$
где мы также строим композицию из $K$ базовых алгоритмов из семейства $\mathbb{B}$:
$$a(x)=a_K(x)=b_1(x)+b_2(x)+\cdots+b_K(x)$$
Выберем в качетсве базовых алгоритмов решающие деревья некоторой фиксированной глубины. Зная методы построения деревьев, обучим алгоритм $b_1(x)$, тогда ошибку предсказания такого алгоритма можем записать в виде:
$$
s_i^1=y_i-b_1(x_i)
$$.

Теперь мы хотим обучить $b_2(x)$ с целью скорректировать $b_1(x)$, причем мы бы хотели, чтобы второй алгоритм предсказывал величины $s_i^1$
$$
a_2(x_i)=b_1(x_i)+b_2(x_i)=b_1(x_i)+s_i^1=b_1(x_i)+(y_i-b_1(x_i))=y_i
$$
Найти совершенный алгоритм скорее всего не получится, но по крайней мере мы можем выбрать из семейства наилучшего представителя для такой задачи. Итак, второе решающее дерево будет обучаться предсказывать разности $s_i^1$:
$$
b_2(x)=\argmin\limits_{b \in\mathbb{B}}L\left(s^1, b(x) \right)
$$

Ожидается, что композиция из двух таких моделей $a_2(x)=b_1(x)+b_2(x)$ станет более качественно предсказывать целевую переменную.

На $k$-ом шаге вычисляется разность между правильным ответом и текущим предсказанием композиции из $k-1$ алгоритмов:
$$
s_i^{k-1}=y_i-\sum\limits_{i=1}^{k-1}b_{k-1}(x_i)=y_i-a_{k-1}(x_i)
$$
Затем $k$-й алгоритм учится предсказывать эту разность:
$$
b_k(x)=\argmin\limits_{b \in\mathbb{B}}L\left(s^{k-1}, b(x) \right)
$$
а композиция в целом обновляется по формуле:
$$
a_k(x)=a_{k-1}(x)+b_k(x)
$$
Обучение $K$ базовых алгоритмов завершает построение композиции.

Стоить упомянуть важное свойство функции потерь. Для этого посчитаем производную функции потерь по предсказанию $z=a_k(x_i)$ модели для $i$-го объекта:
$$
\left.\frac{\partial{L(y_i, z)}}{\partial{z}}\right|_{z=a_k(x_i)}=\left.\frac{\partial}{\partial{z}}\frac{1}{2}\left(y_i-z\right)^2 \right|_{z=a_k(x_i)}=a_k(x_i)-y_i = -s_i^k
$$
Видим, что разность, на которую обучается $k$-й алгоритм, выражается через производную. Таким образом, для каждого объекта $x_i$ очередной алгоритм в бустинге обучается предсказывать антиградиент функции потерь по предсказанию модели $-\frac{\partial L(y_i, z)}{\partial z}$ в точке $a_k(x_i)$ предсказания текущей части композиции на объекте. Эта особенность позволяет обобщить подход построения бустинга на произвольную дифференцируемую функцию потерь. Для этого мы заменяем обучение на разность $s_i^k$ обучением на антиградиент функции потерь ($-g_i^k$), где
$$
\left.g_i^k=\frac{\partial L(y_i, z)}{\partial{z}}\right|_{z=a_k(x_i)}
$$
Обучение композиции можно представить как перемещение предсказания из точки $\left(a_k(x_1), a_k(x_2),\dots,a_k(x_N) \right)$ в точку $\left(a_{k+1}(x_1), a_{k+1}(x_2),\dots,a_{k+1}(x_N) \right)$. В конечном итоге мы ожидаем, что точка  будет располагаться как можно ближе к точке с истинными значениями $\left(y_1, y_2,\dots y_N \right)$.

<center><img src="data//boosting_steps.png" alt="drawing" width="800"/></center>

Изменится ли что-либо в наших действиях, если мы поменяем квадратичную функцию потерь на любую другую? С одной стороны, мы, как и прежде, можем двигаться в направлении уменьшения разности предсказания и истинного значения: любая функция потерь поощряет такие шаги для каждого отдельного объекта, ведь для любой адекватной функции потерь выполнено $L(y, y)=0$. Но мы можем посмотреть на задачу и с другой стороны: не с точки зрения уменьшения расстояния между вектором предсказаний и вектором истинных значений, а с точки зрения уменьшения значения функции потерь. Для наискорейшего уменьшения функции потерь нам необходимо шагнуть в сторону её антиградиента по вектору предсказаний текущей композиции, то есть как раз таки в сторону вектора $\left(-g_1^k,\dots,-g_N^k \right)$. Это направление не обязано совпадать с шагом по направлению уменьшения разности предсказания и истинного значения.

Снова рассмотрим композицию базовых алгоритмов $a(x)=a_K(x)=b_1(x)+b_2(x)+\cdots+b_K(x)$. Строим композицию в виде:
$$
a_k(x)=a_{k-1}(x)+b_k(x)
$$
где вновь добавляемый базовый алгоритм обучается так, чтобы улучшить предсказания текущей композиции:
$$
b_k(x)=\argmin\limits_{b \in\mathbb{B}}L\left(y_i, a_{k-1}(x_i)+b(x_i) \right)
$$
Модель $b_0$ выбирается так, чтобы минимизировать потери на обучающей выборке:
$$
b_0(x)=\argmin\limits_{b \in\mathbb{B}}L\left(y_i, b(x_i) \right)
$$
здесь $L$ - дифференцируемая функция потерь. Для построения базовых алгоритмов на следующих шагах рассмотрим разложение Тейлора функции потерь $L$ до первого члена в окрестности точки $(y_i, a_{k-1}(x_i))$:
$$
\left. L\left(y_i, a_{k-1}(x_i)+b(x_i) \right)\approx L\left(y_i, a_{k-1}(x_i)\right)+b(x_i)\frac{\partial L(y_i, z)}{\partial z}\right|_{z=a_{k-1}(x_i)}=L\left(y_i, a_{k-1}(x_i)\right)+b(x_i)g_i^{k-1}
$$
Избавившись от постоянных членов, мы получим следующую оптимизационную задачу:
$$
b_k(x)\approx\argmin\limits_{b \in\mathbb{B}}\sum\limits_{i=1}^N b(x_i)g_i^{k-1}
$$
Поскольку суммируемое выражение – это скалярное произведение двух векторов, его значение минимизируют $b(x_i)$, пропорциональные значениям $-g_i^{k-1}$. Поэтому на каждой итерации базовые алгоритмы $b_k$ обучаются предсказывать значения антиградиента функции потерь по текущим предсказаниям композиции.

Итак, использованная нами интуиция шага в сторону "уменьшения остатка" привела к оптимальным смещениям в случае квадратичной функции потерь, но для других функций потерь это не так: для них смещение происходит в сторону антиградиента.

**Обучение базового алгоритма**

При построении очередного базового алгоритма $b_{k+1}$ мы решаем задачу регрессии с таргетом, равным антиградиенту функции потерь исходной задачи на предсказании $a_k=b_1+\dots+b_k$.
Теоретически можно воспользоваться любым методом построения регрессионного дерева. Важно выбрать **оценочную функцию** $S$, которая будет показывать, насколько текущая структура дерева хорошо приближает антиградиент. Её нужно будет использовать для построения критерия ветвления.
Алгоритм обучения базового алгоритма:
- по **функции потерь** вычисляется целевая переменная для обучения следующего базового алгоритма:
$$
\left.g_i^k=\frac{\partial L(y_i, z)}{\partial z}\right|_{z=a_k(x_i)}
$$
- строится регрессионное дерево на обучающей выборке $(x_i, -g_i^k)$, минимизирующее выбранную **оценочную функцию**.

Поскольку для построения градиентного бустинга достаточно уметь считать градиент функции потерь по предсказаниям, с его помощью можно решать широкий спектр задач. В библиотеках градиентного бустинга даже реализована возможность создавать свои функции потерь: для этого достаточно уметь вычислять ее градиент, зная истинные значения и текущие предсказания для элементов обучающей выборки.

**Темп обучения**

Обучение композиции с помощью градиентного бустинга может привести к переобучению, если базовые алгоритмы слишком сложные. Например, если сделать решающие деревья слишком глубокими (более 10 уровней), то при обучении бустинга ошибка на обучающей выборке даже при довольно скромном $K$ может приблизиться к нулю, то есть предсказание будет почти идеальным, но на тестовой выборке всё будет плохо.

Существует два решения этой проблемы. *Во-первых*, необходимо упростить базовую модель, уменьшив глубину дерева (либо примерив какие-либо ещё техники регуляризации). *Во-вторых*, мы можем ввести параметр, называемый **темпом обучения (learning rate)** $\eta \in (0, 1]$:
$$
a_{k+1}(x)=a_k(x)+\eta b_{k+1}(x)=b_1(x)+\eta b_2(x)+\eta b_3(x)+\dots+\eta b_{k+1}(x)
$$
Присутствие этого параметра означает, что каждый базовый алгоритм вносит относительно небольшой вклад во всю композицию. Значение параметра обычно определяется эмпирически по входным данным.

**Библиотеки**
- [scikit-learn](https://scikit-learn.org/stable/index.html)
- [LightGBM](https://lightgbm.readthedocs.io/en/latest/)
- [XGBoost](https://xgboost.readthedocs.io/en/latest/)
- [CatBoost](https://catboost.ai/)

Градиентный бустинг используется в очень большом количестве задач. Можно сказать, что это один из двух (наряду с нейронными сетями) подходов к решению задач машинного обучения на практике. Если при работе с текстом, картинками нейронные сети показывают гораздо лучшие результаты, то для табличных данных используют бустинг.